This Python Jupyter Notebook creates the SQL Server tables required to implement the Technical Analysis Measures database as defined in the Entity-Relationship diagram below:

In [1]:
#Import required packages
import pyodbc
import sqlalchemy
from sqlalchemy import create_engine
import urllib
import pandas as pd

In [2]:
#Connect to database 'TAM' using SQlAlchemy
connection_str = "DRIVER={SQL SERVER};SERVER=LAPTOP-BJU2VQE0\SIMPLESERVER;DATABASE=TAM;TRUSTED_CONNECTION=YES"
params = urllib.parse.quote_plus(connection_str)
engine = create_engine('mssql+pyodbc:///?odbc_connect=%s' % params)
conn = engine.connect()

In [3]:
CreateFilePurposes = """CREATE TABLE FilePurposes (FilePurpose VARCHAR(50) PRIMARY KEY NOT NULL)"""

In [4]:
engine.execute(CreateFilePurposes)

In [5]:
CreateSourceFileMeta = """CREATE TABLE SourceFileMetaData
    (
    FilePurpose VARCHAR(50) FOREIGN KEY REFERENCES FilePurposes(FilePurpose) NOT NULL,
    SourceFileID VARCHAR(100) PRIMARY KEY NOT NULL,
    SourceFileName VARCHAR(100) NOT NULL,
    SourceURL VARCHAR(100) NOT NULL,
    SourceDate DATE NOT NULL,
    FileSize VARCHAR(10),
    LocalFilePath VARCHAR(100) NOT NULL
    )"""

In [6]:
engine.execute(CreateSourceFileMeta)

In [7]:
CreateShares = """CREATE TABLE Shares
    (
    OriginalSourceID VARCHAR(100) FOREIGN KEY REFERENCES SourceFileMetaData(SourceFileID) NOT NULL,
    TDIM VARCHAR(4) PRIMARY KEY NOT NULL,
    IssuerName VARCHAR(100) NOT NULL,
    InstrumentName VARCHAR(50) NOT NULL,
    ISIN CHAR(12) NOT NULL,
    MiFIRIdentifierCode CHAR(12) NOT NULL,
    ICBIndustry VARCHAR(50),
    ICBSuperSectorName VARCHAR(50),
    StartDate DATE NOT NULL,
    CountryOfIncorporation VARCHAR(50) NOT NULL,
    TradingCurrency CHAR(3) NOT NULL,
    SecurityMktCap DECIMAL(18,2),
    LSEMarket VARCHAR(50) NOT NULL,
    FCAListingCategory VARCHAR(75),
    MarketSegmentCode CHAR(4) NOT NULL,
    MarketSectorCode VARCHAR(4) NOT NULL
    )"""

In [8]:
engine.execute(CreateShares)

In [9]:
CreateMarketData = """CREATE TABLE MarketData
    (
    MarketDataID AS CONVERT(CHAR(8),MarketDate,112) + TDIM PERSISTED PRIMARY KEY,
    MarketDate DATE NOT NULL,
    TDIM VARCHAR(4) FOREIGN KEY REFERENCES Shares(TDIM) NOT NULL,
    OpenPrice DECIMAL(10,4) NOT NULL,
    ClosePrice DECIMAL(10,4) NOT NULL,
    HighPrice DECIMAL(10,4) NOT NULL,
    LowPrice DECIMAL(10,4) NOT NULL,
    AdjClosePrice DECIMAL(10,4) NOT NULL,
    Volume INTEGER NOT NULL,
    ImportDate DATE NOT NULL,
    ImportAlgorithm VARCHAR(100) NOT NULL
    )"""

In [10]:
engine.execute(CreateMarketData)

In [11]:
CreateTAMs = """CREATE TABLE TechnicalAnalysisMeasures
    (
    Measure VARCHAR(10) PRIMARY KEY NOT NULL,
    FullName VARCHAR(100) NOT NULL
    )"""

In [12]:
engine.execute(CreateTAMs)

In [13]:
CreateTARs = """CREATE TABLE TechnicalAnalysisResults
    (
    TDIM VARCHAR(4) FOREIGN KEY REFERENCES Shares(TDIM) NOT NULL,
    MarketDataID VARCHAR(12) FOREIGN KEY REFERENCES MarketData(MarketDataID) NOT NULL,
    TAMID AS MarketDataID + Measure PERSISTED PRIMARY KEY,
    Measure VARCHAR(10) FOREIGN KEY REFERENCES TechnicalAnalysisMeasures(Measure) NOT NULL,
    AnalysisDate DATE NOT NULL,
    AnalysisAlgorithm VARCHAR(100) NOT NULL,
    AnalysisResult DECIMAL(10,4) NOT NULL
    )"""

In [14]:
engine.execute(CreateTARs)

In [15]:
CreateHoldings = """CREATE TABLE Holdings
    (
    HoldingsID AS CONVERT(CHAR(8),HoldingsDate,112) + TIDM PERSISTED PRIMARY KEY,
    HoldingsDate DATE NOT NULL,
    Amount DECIMAL (10,4) NOT NULL,
    HoldingsAlgorithm VARCHAR(100) NOT NULL,
    HoldingsAlgDate DATE NOT NULL,
    TIDM VARCHAR(4) FOREIGN KEY REFERENCES Shares(TDIM) NOT NULL
    )"""

In [16]:
engine.execute(CreateHoldings)

In [17]:
CreateBuySell = """CREATE TABLE BuySellDecisions
    (
    DecisionID AS CONVERT(CHAR(8),DecisionDate,112) + TAMID + BuySell PERSISTED PRIMARY KEY,
    TIDM VARCHAR(4) FOREIGN KEY REFERENCES Shares(TDIM) NOT NULL,
    DecisionIndicator VARCHAR(10) FOREIGN KEY REFERENCES TechnicalAnalysisMeasures(Measure) NOT NULL,
    TAMID VARCHAR(22) FOREIGN KEY REFERENCES TechnicalAnalysisResults(TAMID) NOT NULL,
    DecisionDate DATE NOT NULL,
    BuySell VARCHAR(4) NOT NULL,
    Price DECIMAL(10,4) NOT NULL,
    DecisionAlgorithm VARCHAR(100) NOT NULL,
    DecisionAlgDate DATE NOT NULL
    )"""

In [18]:
engine.execute(CreateBuySell)

In [19]:
#Get the table names currently in the database
insp = sqlalchemy.inspect(engine)
print(insp.get_table_names())

['BuySellDecisions', 'FilePurposes', 'Holdings', 'MarketData', 'Shares', 'SourceFileMetaData', 'TechnicalAnalysisMeasures', 'TechnicalAnalysisResults']


In [20]:
conn.close()